# Selenium 네이버 뉴스 댓글 스크래핑

## Selenium 및 웹 드라이버 설치

In [62]:
!pip install Selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

import sys
sys.path.insert(0, '/usr/lib/chromium-browser/chromedriver')

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:7 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Fetched 252 kB in 2s (112 kB/s)
Reading package lists... Done
Reading package lists... Done
Building d

## 라이브러리 import

In [63]:
import time
import pandas as pd
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

## `news_scraping()`: 뉴스 스크래핑

In [64]:
def news_scraping(news_url, wd):

  press = wd.find_element_by_xpath('//*[@id="main_content"]/div[1]/div[1]/a/img').get_attribute('title')
  title = wd.find_element_by_id('articleTitle').text
  datetime = wd.find_element_by_class_name('t11').text 
  article = wd.find_element_by_id('articleBodyContents').text

  article = article.replace('// flash 오류를 우회하기 위한 함수 추가','')
  article = article.replace('function _flash_removeCallback() {}','')
  article = article.replace('\n','')
  article = article.replace('\t','')

  good = wd.find_element_by_xpath('//*[@id="spiLayer"]/div[1]/ul/li[1]/a/span[2]').text
  warm = wd.find_element_by_xpath('//*[@id="spiLayer"]/div[1]/ul/li[2]/a/span[2]').text
  sad = wd.find_element_by_xpath('//*[@id="spiLayer"]/div[1]/ul/li[3]/a/span[2]').text
  angry = wd.find_element_by_xpath('//*[@id="spiLayer"]/div[1]/ul/li[4]/a/span[2]').text
  want = wd.find_element_by_xpath('//*[@id="spiLayer"]/div[1]/ul/li[5]/a/span[2]').text
  rec = wd.find_element_by_xpath('//*[@id="toMainContainer"]/a/em[2]').text

  print('뉴스:', [title, press, datetime, article, good, warm, sad, angry, want, rec, news_url])

  return [title, press, datetime, article, good, warm, sad, angry, want, rec, news_url]

## `comments_scraping()`: 뉴스 댓글 스크래핑

In [65]:
def comments_scraping(news_url, wd):

  try:
    wd.find_element_by_class_name('u_cbox_btn_view_comment').click()
    print('[댓글 더보기]', end='')
    time.sleep(1)

    while True:
      wd.find_element_by_class_name('u_cbox_btn_more').click()
      print('[더보기]', end='')
      time.sleep(1)
  except:
    pass

    btn_reply_list = wd.find_elements_by_class_name('u_cbox_btn_reply')
    for btn_reply in btn_reply_list:
        btn_reply.send_keys('\n')
        print('[답글]', end='')
        time.sleep(1)

    print('[댓글 스크래핑]')
    comments_idx = 0
    comments_df = pd.DataFrame(columns=('Contents', 'Name', 'Datetime', 'Recommned', 'Unrecommned', 'URL'))
    comments = wd.find_elements_by_class_name('u_cbox_comment_box')
    for comment in comments:
      try:
        name = comment.find_element_by_class_name('u_cbox_nick').text
        date = comment.find_element_by_class_name('u_cbox_date').text
        contents = comment.find_element_by_class_name('u_cbox_contents').text
        recomm = comment.find_element_by_class_name('u_cbox_cnt_recomm').text
        unrecomm = comment.find_element_by_class_name('u_cbox_cnt_unrecomm').text
        print(f' 댓글 #{comments_idx+1}:',[contents, name, date, recomm, unrecomm, news_url])
        comments_df.loc[comments_idx] = [contents, name, date, recomm, unrecomm, news_url]
        comments_idx += 1
      except NoSuchElementException:
        #print('[삭제되거나 부적절한 댓글]')
        continue

    return comments_df

## `scraping()`: 스크래핑 함수

In [66]:
def scraping(news_max=3):
  wd = webdriver.Chrome('chromedriver',options=chrome_options)
  wd.implicitly_wait(3)

  wd.execute_script('window.open("about:blank","_blank");')
  tabs = wd.window_handles

  wd.switch_to.window(tabs[0])
  query = input('검색어 입력: ')
  search_url = 'https://search.naver.com/search.naver?where=news&ie=utf8&sm=nws_hty&query=' + query
  wd.get(search_url)

  news_idx = 0
  news_df = pd.DataFrame(columns=('Title', 'Press', 'DataTime', 'Article', 'Good', 'Warm', 'Sad', 'Angry', 'Want', 'Recommend', 'URL'))
  comments_df = pd.DataFrame()

  while True:
    inline_list = wd.find_elements_by_class_name('txt_inline')
    for inline in inline_list:
      try:
        news_url = inline.find_element_by_class_name('_sp_each_url').get_attribute('href')
      except:
        continue

      wd.switch_to.window(tabs[1])
      wd.get(news_url)

      news_df.loc[news_idx] = news_scraping(news_url, wd)
      news_idx += 1

      df = comments_scraping(news_url, wd)
      comments_df = pd.concat([comments_df, df])

      if news_idx >= news_max:
        break

    if news_idx >= news_max:
        break

    try:
      wd.switch_to.window(tabs[0])
      wd.find_element_by_class_name('next').click()
      time.sleep(1)
    except:
      break

  wd.close()

  return news_df, comments_df

## 스크래핑 실행

In [67]:
news_df, comments_df = scraping()

검색어 입력: 인공지능
뉴스: ['교원그룹-셀바스AI ‘인공지능 기반 디지털 헬스케어’ 업무협약', '스포츠동아', '2020.08.30. 오전 10:15', '장동하 교원그룹 기획조정실장(왼쪽)과 곽민철 셀바스AI 대표이사가 업무협약 후 기념촬영을 하고 있다. 사진제공 l 교원그룹교원그룹이 인공지능(AI) 업체 셀바스AI와 ‘인공지능 기반 디지털 헬스케어 사업 협력을 위한 업무협약’을 체결했다.이번 협약을 통해 교원그룹 웰스의 건강가전 브랜드와 셀바스AI의 인공지능 헬스케어 솔루션 ‘셀비 체크업’을 연계한 생활 맞춤 솔루션 개발, 융합 데이터 활용 건강 예측 모델 및 기술 개발 등 인공지능-디지털 헬스케어 구축에 힘을 모으기로 했다. 이용자 생활패턴, 건강상태 등을 분석하고 고도화된 생활 맞춤 솔루션을 제공한다는 게 교원그룹 측 설명이다.정정욱 기자 jjay@donga.com▶ 연예현장 진짜 이야기 / 스타들의 아찔한 순간▶ 나의 아이돌을 픽♥ 해주세요. / 이번 주는 누가 1위?- Copyrights ⓒ 스포츠동아 무단 전재 및 재배포 금지 -', '0', '0', '0', '0', '0', '', 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=382&aid=0000850697']
[댓글 스크래핑]
뉴스: ['구직자 48.1% "인공지능 면접이 사람보다 더 공정해"', '뉴시스', '2020.08.28. 오전 8:42', '면접 선호도는 대면면접(46.8%)이 AI면접(16.2%) 보다 높아구직자 40.2% "인공지능이 나를 평가하는 것에 거부감 없다"[서울=뉴시스] 고은결 기자 = 최근 인공지능(AI) 면접을 도입하는 기업이 늘고 있는 가운데, 구직자 절반 가량은 \'대면면접\'보다 \'AI 면접\'이 더 공정하다고 생각하는 것으로 나타났다.잡코리아는 상반기 취업활동을 한 구직자 1688명을 대상으로 면접 선호 유형에 대해 조사를 진행한 결과, 아직까지 구직자들은 인공지능이 사람을

In [68]:
news_df

,Title,Press,DataTime,Article,Good,Warm,Sad,Angry,Want,Recommend,URL
0,교원그룹-셀바스AI ‘인공지능 기반 디지털 헬스케어’ 업무협약,스포츠동아,2020.08.30. 오전 10:15,장동하 교원그룹 기획조정실장(왼쪽)과 곽민철 셀바스AI 대표이사가 업무협약 후 기념...,0,0,0,0,0,,https://news.naver.com/main/read.nhn?mode=LSD&...
1,"구직자 48.1% ""인공지능 면접이 사람보다 더 공정해""",뉴시스,2020.08.28. 오전 8:42,면접 선호도는 대면면접(46.8%)이 AI면접(16.2%) 보다 높아구직자 40.2...,1,1,0,0,0,1,https://news.naver.com/main/read.nhn?mode=LSD&...
2,인공지능 기반 자율방역 로봇 등 40개 개발과제에 370억원 지원,연합뉴스,2020.08.25. 오후 12:00,과기부 '비대면 비즈니스 디지털혁신 기술개발' 과제 40개 선정(서울=연합뉴스) 정...,0,0,0,0,0,,https://news.naver.com/main/read.nhn?mode=LSD&...


In [69]:
comments_df

,Contents,Name,Datetime,Recommned,Unrecommned,URL
0,대통령도 인공지능이 차라리 더 낫겠지?.....인공지능이 더 합리적이고 이성적이고 ...,mega****,2020.08.28. 00:21,0,0,https://news.naver.com/main/read.nhn?mode=LSD&...
1,여러 실험에서 인상 보고 느낌 보고 첫인상 보고 뽑은 사람이 모든 걸 배제하고 여러...,v6ma****,2020.08.28. 00:10,0,0,https://news.naver.com/main/read.nhn?mode=LSD&...
2,"AI도 결국 사람이 처음에 프로그램을 입력해야 하는데, 그 입력 과정에서 불공정한 ...",snak****,2020.08.28. 00:08,0,0,https://news.naver.com/main/read.nhn?mode=LSD&...
